In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import io

# load total reads

In [51]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/reports/seqkit/raw.fastq.report.txt"

reads = pd.read_csv(fpath, sep=r"\s+")
reads['num_seqs'] = reads['num_seqs'].astype(str).str.replace(',', '').astype(int)
reads['sample'] = reads['file'].apply(lambda x: x[:2])
reads['cell'] = reads['file'].apply(lambda x: x[2:5])

cell_ids = tuple(zip(reads['sample'].values, reads['cell'].values,))
read_mapper = dict(zip(cell_ids, reads['num_seqs'].values))

print('done')

done


# Load all

In [82]:
dpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/barcode_locations/"

bins = [0, 1, 3, 4, float('inf')]
category_labels = ["1", "2", "3", "4+"]

df = []

for f in os.listdir(dpath):
    fpath = f"{dpath}{f}"
    tmp_raw = pd.read_csv(fpath, sep='\t')
    tmp = tmp_raw.copy()
    tmp['sample'] = f.replace(".csv", "")[0:2]
    tmp['cell'] = f.replace(".csv", "")[2:]

    # only consider when the barcode is the correct one
    tmp = tmp[tmp['cell'] == tmp['patternName']]

    # aggregate counts by read
    tmp = tmp.groupby(['sample', 'strand', 'cell', 'seqID']).agg(
        n_hits = ('start', 'nunique'),
    ).reset_index()

    # set categories
    tmp['barcode_hits'] = pd.cut(tmp['n_hits'], bins=bins, labels=category_labels)

    # agg by  cell
    tmp = tmp.groupby(['sample', 'strand', 'cell', 'barcode_hits']).agg(
        hits = ('n_hits', 'count'),
    ).reset_index()

    # pivot
    tmp = pd.pivot_table(tmp, index=['sample', 'cell'],
                         columns=['strand', 'barcode_hits'],
                         values='hits')
    tmp['total_reads'] = tmp.index.map(read_mapper)
    tmp['percent_positive_strand'] = tmp["+"].sum(axis=1) / tmp['total_reads']
    tmp['percent_negative_strand'] = tmp["-"].sum(axis=1) / tmp['total_reads']

    if tmp['percent_negative_strand'].values[0] > 1:
        print(f)
        df.append(tmp)
        break
        
    df.append(tmp)

df = pd.concat(df)
df = df.reset_index()
print(f"{df.shape=}")
df

df.shape=(284, 13)


strand       sample cell       +                             -                 \
barcode_hits                   1       2      3     4+       1       2      3   
0                o3  b93  139818     590      0      0  129809     677      0   
1                o3  b91   36155     123      0      1   26259      95      0   
2                o3  b49   75874     866    244    394   35110     586    106   
3                o2  b66   10157   13677   3565   5049    7749   11526   3945   
4                o2  b94   38316   57007  10770  12881   37249   48694  11518   
..              ...  ...     ...     ...    ...    ...     ...     ...    ...   
279              o3  b32   45487     181      2      3   39411     140      2   
280              o3  b09  178679  112367  38347  60600  175313   95512  28037   
281              o3  b66  116423    3898   1338   1909   62927    2429    450   
282              o3  b03  156507    1003     61    102   86522     500     32   
283              o3  b95  134837  122323  33511  56209   96087  115358  39800   

strand              total_reads percent_positive_strand  \
barcode_hits     4+                                       
0                 0      191110                0.734697   
1                 1       45619                0.795261   
2               195      105650                0.732399   
3              8783       44272                0.732924   
4             13287      145024                0.820375   
..              ...         ...                     ...   
279               0       59352                0.769528   
280           46665      449123                0.868343   
281             649      175807                0.702862   
282              51      254770                0.618884   
283           79348      472468                0.734187   

strand       percent_negative_strand  
barcode_hits                          
0                           0.682780  
1                           0.577720  
2                           0.340719  
3                           0.722872  
4                           0.763653  
..                               ...  
279                         0.666414  
280                         0.769337  
281                         0.378000  
282                         0.341897  
283                         0.699715  

[284 rows x 13 columns]

In [83]:
outpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/barcode_search_summary.xlsx"
with pd.ExcelWriter(outpath, engine='xlsxwriter') as writer:

    # save each sample to a new tab
    for sample, group in df.groupby('sample'):
        print(sample)
        group.to_excel(writer, sheet_name=f"Sample_{sample}", index=True)

print(f'saved: {outpath}')

o1
o2
o3
saved: /scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/barcode_search_summary.xlsx


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
break